In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
from datetime import timedelta
from typing import List, Dict
import random

import pandas as pd
pd.set_option('display.max_rows', 200)
import numpy as np

from tqdm import tqdm

import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

from data_handlers.runtimes import *

In [2]:
experiment_path = pathlib.Path(f"/home/sterz/maci-docker-compose/maci_data/binary_files")
experiment_ids = [25, 26, 27, 28, 33]

In [3]:
event_frame = parse_bundle_events(experiment_path, experiment_ids)

Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/25/2140
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/25/2137
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/25/2091
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/25/2105
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/25/2075
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/25/2153
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/25/2124
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/25/2088
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/25/2066
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/25/2082
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/25/2116
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/25/2061
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/25/2111
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/25/2085
Parsin

In [21]:
event_frame["Software/CLA"] = event_frame['Software'] + "/" + event_frame['CLA']

In [24]:
creation_instances = event_frame[event_frame.Event == "creation"].groupby("Simulation ID")
delivery_instances = event_frame[event_frame.Event == "delivery"]

In [25]:
delivered_list = []
runtime_list = []
for sim_instance_id, creations in tqdm(creation_instances):
    deliveries_sid = delivery_instances[delivery_instances["Simulation ID"] == sim_instance_id]
    
    for _, row in creations.iterrows():
        # Compute deliverys
        bundle = row.Bundle
        deliveries = deliveries_sid[deliveries_sid.Bundle == bundle]

        was_delivered = not deliveries.empty

        row_dict = {
            "Simulation ID": sim_instance_id,
            "Bundle": bundle,
            "Delivered": was_delivered,
            "CLA": row.CLA,
            "Software/CLA": row["Software/CLA"],
            "Loss": row.Loss,
            "Payload Size": row["Payload Size"],
            "Software": row.Software,
            "# Nodes": row["# Nodes"],
            "# Payloads": row["# Payloads"],
        }
        delivered_list.append(row_dict)        
        
        # Compute runtimes
        if was_delivered:
            delivery_time = deliveries["Timestamp"].min()
            creation_time = row["Timestamp"]
            runtime = delivery_time - creation_time
            
            runtime_dict = {
                "Simulation ID": sim_instance_id,
                "Bundle": bundle,
                "Runtime": (runtime.to_pytimedelta() / timedelta(milliseconds=1)) / 1000,
                "CLA": row.CLA,
                "Software/CLA": row["Software/CLA"],
                "Loss": row.Loss,
                "Software": row.Software,
                "# Nodes": row["# Nodes"],
                "Payload Size": row["Payload Size"],
                "# Payloads": row["# Payloads"],
            }
            runtime_list.append(runtime_dict)
        
delivered_df = pd.DataFrame(delivered_list)
runtime_df = pd.DataFrame(runtime_list)

100%|██████████| 432/432 [00:53<00:00,  8.15it/s]


In [26]:
row_dicts = []

for sim_instance_id, bundles in delivered_df.groupby("Simulation ID"):
    total_bundles = len(bundles.index)
    delivered_bundles = len(bundles[bundles.Delivered == True].index)
    
    cla = bundles["CLA"].values[0]
    software_cla = bundles["Software/CLA"].values[0]
    loss = bundles["Loss"].values[0]
    software = bundles["Software"].values[0]
    nodes = bundles["# Nodes"].values[0]
    payload_size = bundles["Payload Size"].values[0]
    payloads = bundles["# Payloads"].values[0]
    
    row_dict = {
        "Simulation ID": sim_instance_id,
        "Delivered (%)": (float(delivered_bundles) / float(total_bundles)) * 100,
        "CLA": cla,
        "Software/CLA": software_cla,
        "Loss": loss,
        "Software": software,
        "# Nodes": nodes,
        "Payload Size": payload_size,
        "# Payloads": payloads,
    }
    
    row_dicts.append(row_dict)

ratio_df = pd.DataFrame.from_dict(row_dicts)

In [27]:
runtime_norm = pd.DataFrame()
for sim_id, simulation in runtime_df.groupby("Simulation ID"):
    simulation["Runtime (norm)"] = simulation["Runtime"] / ratio_df[ratio_df["Simulation ID"] == sim_id]["Delivered (%)"].iloc[0]
    runtime_norm = pd.concat([runtime_norm, simulation])

In [36]:
fig = px.box(
    runtime_df[runtime_df["# Nodes"] == 32],
    x="Software/CLA", 
    y="Runtime",
    color="Loss",
    facet_col="# Payloads",
    facet_row="Payload Size",
    category_orders={
        "Loss": [0, 10, 30, 50],
        "Payload Size": [1000, 100000, 1000000]
    },
)

fig.show()

In [34]:
fig = px.box(
    runtime_df[runtime_df["# Nodes"] == 2],
    x="Software/CLA", 
    y="Runtime",
    color="Loss",
    facet_col="# Payloads",
    facet_row="Payload Size",
    category_orders={
        "Loss": [0, 10, 30, 50],
        "Payload Size": [1000, 100000, 1000000]
    },
)

fig.show()

In [37]:
fig = px.histogram(
    ratio_df[ratio_df["# Nodes"] == 2],
    x="Software/CLA",
    y="Delivered (%)",
    color="Loss",
    facet_col="# Payloads",
    facet_row="Payload Size",
    barmode='group',
    histfunc='avg',
    category_orders={
        "Loss": [0, 10, 30, 50],
        "Payload Size": [1000, 100000, 1000000]
    },
)
    
fig.update_xaxes(type='category')
fig.show()

In [38]:
fig = px.histogram(
    ratio_df[ratio_df["# Nodes"] == 32],
    x="Software/CLA",
    y="Delivered (%)",
    color="Loss",
    facet_col="# Payloads",
    facet_row="Payload Size",
    barmode='group',
    histfunc='avg',
    category_orders={
        "Loss": [0, 10, 30, 50],
        "Payload Size": [1000, 100000, 1000000]
    },
)
    
fig.update_xaxes(type='category')
fig.show()

In [25]:
for (sim_instance_id, bundle_id), bundles in event_frame.groupby(["Simulation ID", "Bundle"]):
    print(bundles)
    break

       Simulation ID  Payload Size               Timestamp      Event Node  \
787034          1309       1000000 1900-01-01 14:27:11.355   creation   n1   
787036          1309       1000000 1900-01-01 14:27:11.372    sending   n1   
787035          1309       1000000 1900-01-01 14:27:11.372    sending   n1   
786726          1309       1000000 1900-01-01 14:27:11.914  reception   n2   
786727          1309       1000000 1900-01-01 14:27:12.024   delivery   n2   
786728          1309       1000000 1900-01-01 14:27:12.139  reception   n2   

                          Bundle Software  Bundles per Second    CLA  Loss  \
787034  dtn://n1/-730909630708-0   DTN7Go                 100  quicl     0   
787036  dtn://n1/-730909630708-0   DTN7Go                 100  quicl     0   
787035  dtn://n1/-730909630708-0   DTN7Go                 100  quicl     0   
786726  dtn://n1/-730909630708-0   DTN7Go                 100  quicl     0   
786727  dtn://n1/-730909630708-0   DTN7Go                 100  